# Content based recommendation system

*Draft*

Video 1: https://www.youtube.com/watch?v=hOQg2LQM4ec&ab_channel=MiningMassiveDatasets \
Notes: 
- It would be interesting to mention the "long tail" phenomenom to explain why recommendation systems are interesting
- The key to recommendations is to have the utility matrix and predict the values that don't appear there

Video 2 (Content based): https://www.youtube.com/watch?v=IlqnNWuqToo&ab_channel=MiningMassiveDatasets \

- Main idea of content based: recommend items to customer x similar to previous items rated highly by x

- Plan of action: start with a user and find out a set of items the user likes using explicit and implicit data -> we are going to build an item profile -> Infer user profile -> Match -> Recommemd

1. For each item, create an item profile. Profile is a set of features. Convenient to think it as a vector
2. User profile: (weighted) average of rated item profiles

### Read the data

Useful features:
- artist_terms (tags)
- similar_artists?
- time_signature
- year
- artist_mbtags
- mode

In [1]:
import pandas as pd
import os
import hdf5_getters as hdf5_getters

path = 'MillionSongSubset'

In [2]:
songs_list = []

for (root, dirs, file) in os.walk(path):
    for f in file:
        songs_list.append(os.path.dirname(f))

            
print(len(songs_list))

10000


In [3]:
## READ DATA PATH FROM FILE
songs_file_paths = []

for root, dirs, files in os.walk(os.path.abspath(path)):
    for file in files:        
        strs = os.path.join(root, file)
        new_strs = strs.replace('\\','/')
        songs_file_paths.append(new_strs)

In [4]:
print(songs_file_paths[0])

/Users/alejandranavarrocastillo/Documents/GitHub/comp_tools/MillionSongSubset/A/R/R/TRARRZU128F4253CA2.h5


In [5]:
len(songs_file_paths)

10000

In [10]:
### Create pandas table

data = []

for i in range(100):
    record = []
    
    #open specific song path
    h5 = hdf5_getters.open_h5_file_read(songs_file_paths[i])
    
    #getters
    artist_id = hdf5_getters.get_artist_id(h5)
    artist_id = artist_id.decode("utf-8")     
    
    artist_name  = hdf5_getters.get_artist_name(h5)
    artist_name = artist_name.decode("utf-8") 
    
    artist_location  = hdf5_getters.get_artist_location(h5)
    artist_location = artist_location.decode("utf-8") 
    
    song_id = hdf5_getters.get_song_id(h5)
    print(type(song_id), song_id)
    song_id = song_id.decode("utf-8")
    print(song_id)
    
    song_name = hdf5_getters.get_title(h5)
    song_name = song_name.decode("utf-8")
    
    song_hottness = hdf5_getters.get_song_hotttnesss(h5)
    
    time_signature = hdf5_getters.get_time_signature(h5)
    
    artist_terms = hdf5_getters.get_artist_terms(h5)
    print(artist_terms[0])
    for j in range(len(artist_terms)):
        print('\n', type(artist_terms[j]), artist_terms[j])
        artist_terms[j] = artist_terms[j].decode("utf-8")
        print(type(artist_terms[j]), artist_terms[j])
    #print(artist_terms)
    
    artist_mbtags = hdf5_getters.get_artist_mbtags(h5)
    
    mode = hdf5_getters.get_mode(h5)
    
    year = hdf5_getters.get_year(h5)
    
    #close file
    h5.close()
    
    record.append(artist_id)
    record.append(artist_name)
    record.append(artist_location)
    record.append(song_id)
    record.append(song_name)
    record.append(song_hottness)
    record.append(time_signature)
    record.append(artist_terms)
    record.append(artist_mbtags)
    record.append(mode)
    record.append(year)
    
    #add
    data.append(record)

<class 'numpy.bytes_'> b'SOGSMXL12A81C23D88'
SOGSMXL12A81C23D88
b'chanson'

 <class 'numpy.bytes_'> b'chanson'
<class 'numpy.bytes_'> b'chanson'

 <class 'numpy.bytes_'> b'visual kei'
<class 'numpy.bytes_'> b'visual kei'

 <class 'numpy.bytes_'> b'hip hop'
<class 'numpy.bytes_'> b'hip hop'

 <class 'numpy.bytes_'> b'pop rock'
<class 'numpy.bytes_'> b'pop rock'

 <class 'numpy.bytes_'> b'british pop'
<class 'numpy.bytes_'> b'british pop'

 <class 'numpy.bytes_'> b'jrock'
<class 'numpy.bytes_'> b'jrock'

 <class 'numpy.bytes_'> b'world music'
<class 'numpy.bytes_'> b'world music'

 <class 'numpy.bytes_'> b'downtempo'
<class 'numpy.bytes_'> b'downtempo'

 <class 'numpy.bytes_'> b'singer-songwriter'
<class 'numpy.bytes_'> b'singer-songwriter'

 <class 'numpy.bytes_'> b'latin pop'
<class 'numpy.bytes_'> b'latin pop'

 <class 'numpy.bytes_'> b'french'
<class 'numpy.bytes_'> b'french'

 <class 'numpy.bytes_'> b'male vocalist'
<class 'numpy.bytes_'> b'male vocalist'

 <class 'numpy.bytes_'> b'

<class 'numpy.bytes_'> b'SOBTEHX12A6D4FBF18'
SOBTEHX12A6D4FBF18
b'post-hardcore'

 <class 'numpy.bytes_'> b'post-hardcore'
<class 'numpy.bytes_'> b'post-hardcore'

 <class 'numpy.bytes_'> b'alternative metal'
<class 'numpy.bytes_'> b'alternative metal'

 <class 'numpy.bytes_'> b'metalcore'
<class 'numpy.bytes_'> b'metalcore'

 <class 'numpy.bytes_'> b'heavy metal'
<class 'numpy.bytes_'> b'heavy metal'

 <class 'numpy.bytes_'> b'death metal'
<class 'numpy.bytes_'> b'death metal'

 <class 'numpy.bytes_'> b'rock'
<class 'numpy.bytes_'> b'rock'

 <class 'numpy.bytes_'> b'metal'
<class 'numpy.bytes_'> b'metal'

 <class 'numpy.bytes_'> b'emo'
<class 'numpy.bytes_'> b'emo'

 <class 'numpy.bytes_'> b'screamo'
<class 'numpy.bytes_'> b'screamo'

 <class 'numpy.bytes_'> b'soundtrack'
<class 'numpy.bytes_'> b'soundtrack'

 <class 'numpy.bytes_'> b'melodic'
<class 'numpy.bytes_'> b'melodic'

 <class 'numpy.bytes_'> b'guitar'
<class 'numpy.bytes_'> b'guitar'

 <class 'numpy.bytes_'> b'heavy'
<class 

<class 'numpy.bytes_'> b'SOAMBCI12AC4685087'
SOAMBCI12AC4685087
b'french pop'

 <class 'numpy.bytes_'> b'french pop'
<class 'numpy.bytes_'> b'french pop'
<class 'numpy.bytes_'> b'SOQNRHE12A8AE4849A'
SOQNRHE12A8AE4849A
b'space age pop'

 <class 'numpy.bytes_'> b'space age pop'
<class 'numpy.bytes_'> b'space age pop'

 <class 'numpy.bytes_'> b'bossa nova'
<class 'numpy.bytes_'> b'bossa nova'

 <class 'numpy.bytes_'> b'jazz'
<class 'numpy.bytes_'> b'jazz'

 <class 'numpy.bytes_'> b'lounge'
<class 'numpy.bytes_'> b'lounge'

 <class 'numpy.bytes_'> b'swing'
<class 'numpy.bytes_'> b'swing'

 <class 'numpy.bytes_'> b'big band'
<class 'numpy.bytes_'> b'big band'

 <class 'numpy.bytes_'> b'mondiovision'
<class 'numpy.bytes_'> b'mondiovision'

 <class 'numpy.bytes_'> b'instrumental'
<class 'numpy.bytes_'> b'instrumental'

 <class 'numpy.bytes_'> b'american'
<class 'numpy.bytes_'> b'american'

 <class 'numpy.bytes_'> b'modern jazz'
<class 'numpy.bytes_'> b'modern jazz'
<class 'numpy.bytes_'> b'SO

 <class 'numpy.bytes_'> b'oldies'
<class 'numpy.bytes_'> b'oldies'

 <class 'numpy.bytes_'> b'trance'
<class 'numpy.bytes_'> b'trance'

 <class 'numpy.bytes_'> b'pop rock'
<class 'numpy.bytes_'> b'pop rock'

 <class 'numpy.bytes_'> b'electronic'
<class 'numpy.bytes_'> b'electronic'

 <class 'numpy.bytes_'> b'ska'
<class 'numpy.bytes_'> b'ska'

 <class 'numpy.bytes_'> b'techno'
<class 'numpy.bytes_'> b'techno'

 <class 'numpy.bytes_'> b'reggae'
<class 'numpy.bytes_'> b'reggae'

 <class 'numpy.bytes_'> b'blues'
<class 'numpy.bytes_'> b'blues'

 <class 'numpy.bytes_'> b'rock'
<class 'numpy.bytes_'> b'rock'

 <class 'numpy.bytes_'> b'west coast rap'
<class 'numpy.bytes_'> b'west coast rap'

 <class 'numpy.bytes_'> b'hip hop'
<class 'numpy.bytes_'> b'hip hop'

 <class 'numpy.bytes_'> b'rap'
<class 'numpy.bytes_'> b'rap'

 <class 'numpy.bytes_'> b'hardcore'
<class 'numpy.bytes_'> b'hardcore'

 <class 'numpy.bytes_'> b'punk'
<class 'numpy.bytes_'> b'punk'

 <class 'numpy.bytes_'> b'male vocal

 <class 'numpy.bytes_'> b'roots reggae'
<class 'numpy.bytes_'> b'roots reggae'

 <class 'numpy.bytes_'> b'rock steady'
<class 'numpy.bytes_'> b'rock steady'

 <class 'numpy.bytes_'> b'dancehall'
<class 'numpy.bytes_'> b'dancehall'

 <class 'numpy.bytes_'> b'dub'
<class 'numpy.bytes_'> b'dub'

 <class 'numpy.bytes_'> b'ska'
<class 'numpy.bytes_'> b'ska'

 <class 'numpy.bytes_'> b'classic'
<class 'numpy.bytes_'> b'classic'

 <class 'numpy.bytes_'> b'jazz'
<class 'numpy.bytes_'> b'jazz'

 <class 'numpy.bytes_'> b'political'
<class 'numpy.bytes_'> b'political'

 <class 'numpy.bytes_'> b'instrumental'
<class 'numpy.bytes_'> b'instrumental'

 <class 'numpy.bytes_'> b'guitar'
<class 'numpy.bytes_'> b'guitar'

 <class 'numpy.bytes_'> b'rock'
<class 'numpy.bytes_'> b'rock'

 <class 'numpy.bytes_'> b'pop'
<class 'numpy.bytes_'> b'pop'

 <class 'numpy.bytes_'> b'acoustic'
<class 'numpy.bytes_'> b'acoustic'

 <class 'numpy.bytes_'> b'smooth'
<class 'numpy.bytes_'> b'smooth'

 <class 'numpy.bytes_'

...

b'british pop'

 <class 'numpy.bytes_'> b'british pop'
<class 'numpy.bytes_'> b'british pop'

 <class 'numpy.bytes_'> b'pop rock'
<class 'numpy.bytes_'> b'pop rock'

 <class 'numpy.bytes_'> b'oldies'
<class 'numpy.bytes_'> b'oldies'

 <class 'numpy.bytes_'> b'new wave'
<class 'numpy.bytes_'> b'new wave'

 <class 'numpy.bytes_'> b'soundtrack'
<class 'numpy.bytes_'> b'soundtrack'

 <class 'numpy.bytes_'> b'male vocalist'
<class 'numpy.bytes_'> b'male vocalist'

 <class 'numpy.bytes_'> b'singer-songwriter'
<class 'numpy.bytes_'> b'singer-songwriter'

 <class 'numpy.bytes_'> b'dance'
<class 'numpy.bytes_'> b'dance'

 <class 'numpy.bytes_'> b'chill-out'
<class 'numpy.bytes_'> b'chill-out'

 <class 'numpy.bytes_'> b'rock'
<class 'numpy.bytes_'> b'rock'

 <class 'numpy.bytes_'> b'jazz'
<class 'numpy.bytes_'> b'jazz'

 <class 'numpy.bytes_'> b'scottish'
<class 'numpy.bytes_'> b'scottish'

 <class 'numpy.bytes_'> b'pop'
<class 'numpy.bytes_'> b'pop'

 <class 'numpy.bytes_'> b'80s'
<class 'numpy

b'death core'

 <class 'numpy.bytes_'> b'death core'
<class 'numpy.bytes_'> b'death core'

 <class 'numpy.bytes_'> b'technical death metal'
<class 'numpy.bytes_'> b'technical death metal'

 <class 'numpy.bytes_'> b'brutal death metal'
<class 'numpy.bytes_'> b'brutal death metal'

 <class 'numpy.bytes_'> b'metalcore'
<class 'numpy.bytes_'> b'metalcore'

 <class 'numpy.bytes_'> b'thrash metal'
<class 'numpy.bytes_'> b'thrash metal'

 <class 'numpy.bytes_'> b'death metal'
<class 'numpy.bytes_'> b'death metal'

 <class 'numpy.bytes_'> b'black metal'
<class 'numpy.bytes_'> b'black metal'

 <class 'numpy.bytes_'> b'grindcore'
<class 'numpy.bytes_'> b'grindcore'

 <class 'numpy.bytes_'> b'metal'
<class 'numpy.bytes_'> b'metal'

 <class 'numpy.bytes_'> b'hardcore'
<class 'numpy.bytes_'> b'hardcore'

 <class 'numpy.bytes_'> b'sludge'
<class 'numpy.bytes_'> b'sludge'

 <class 'numpy.bytes_'> b'grind'
<class 'numpy.bytes_'> b'grind'

 <class 'numpy.bytes_'> b'doom metal'
<class 'numpy.bytes_'> b'

b'nu metal'

 <class 'numpy.bytes_'> b'nu metal'
<class 'numpy.bytes_'> b'nu metal'

 <class 'numpy.bytes_'> b'rapcore'
<class 'numpy.bytes_'> b'rapcore'

 <class 'numpy.bytes_'> b'modern rock'
<class 'numpy.bytes_'> b'modern rock'

 <class 'numpy.bytes_'> b'industrial metal'
<class 'numpy.bytes_'> b'industrial metal'

 <class 'numpy.bytes_'> b'alternative metal'
<class 'numpy.bytes_'> b'alternative metal'

 <class 'numpy.bytes_'> b'hard rock'
<class 'numpy.bytes_'> b'hard rock'

 <class 'numpy.bytes_'> b'alternative rock'
<class 'numpy.bytes_'> b'alternative rock'

 <class 'numpy.bytes_'> b'rock'
<class 'numpy.bytes_'> b'rock'

 <class 'numpy.bytes_'> b'alternative'
<class 'numpy.bytes_'> b'alternative'

 <class 'numpy.bytes_'> b'metal'
<class 'numpy.bytes_'> b'metal'

 <class 'numpy.bytes_'> b'industrial'
<class 'numpy.bytes_'> b'industrial'

 <class 'numpy.bytes_'> b'american'
<class 'numpy.bytes_'> b'american'

 <class 'numpy.bytes_'> b'90s'
<class 'numpy.bytes_'> b'90s'

 <class '

<class 'numpy.bytes_'> b'alternative rock'

 <class 'numpy.bytes_'> b'new wave'
<class 'numpy.bytes_'> b'new wave'

 <class 'numpy.bytes_'> b'guitar'
<class 'numpy.bytes_'> b'guitar'

 <class 'numpy.bytes_'> b'melancholia'
<class 'numpy.bytes_'> b'melancholia'

 <class 'numpy.bytes_'> b'romantic'
<class 'numpy.bytes_'> b'romantic'

 <class 'numpy.bytes_'> b'soft'
<class 'numpy.bytes_'> b'soft'

 <class 'numpy.bytes_'> b'song writer'
<class 'numpy.bytes_'> b'song writer'

 <class 'numpy.bytes_'> b'indie'
<class 'numpy.bytes_'> b'indie'

 <class 'numpy.bytes_'> b'dreamy'
<class 'numpy.bytes_'> b'dreamy'

 <class 'numpy.bytes_'> b'acoustic'
<class 'numpy.bytes_'> b'acoustic'

 <class 'numpy.bytes_'> b'beautiful'
<class 'numpy.bytes_'> b'beautiful'

 <class 'numpy.bytes_'> b'twee'
<class 'numpy.bytes_'> b'twee'

 <class 'numpy.bytes_'> b'sad'
<class 'numpy.bytes_'> b'sad'
<class 'numpy.bytes_'> b'SOBDENU12CF5465247'
SOBDENU12CF5465247
b'hardcore punk'

 <class 'numpy.bytes_'> b'hardcore pu

 <class 'numpy.bytes_'> b'beats'
<class 'numpy.bytes_'> b'beats'

 <class 'numpy.bytes_'> b'female'
<class 'numpy.bytes_'> b'female'

 <class 'numpy.bytes_'> b'voice'
<class 'numpy.bytes_'> b'voice'

 <class 'numpy.bytes_'> b'producer'
<class 'numpy.bytes_'> b'producer'

 <class 'numpy.bytes_'> b'united states'
<class 'numpy.bytes_'> b'united states'
<class 'numpy.bytes_'> b'SOVKSAD12A8C142C13'
SOVKSAD12A8C142C13
b'funk rock'

 <class 'numpy.bytes_'> b'funk rock'
<class 'numpy.bytes_'> b'funk rock'

 <class 'numpy.bytes_'> b'chill-out'
<class 'numpy.bytes_'> b'chill-out'

 <class 'numpy.bytes_'> b'jungle music'
<class 'numpy.bytes_'> b'jungle music'

 <class 'numpy.bytes_'> b'acid jazz'
<class 'numpy.bytes_'> b'acid jazz'

 <class 'numpy.bytes_'> b'jazz fusion'
<class 'numpy.bytes_'> b'jazz fusion'

 <class 'numpy.bytes_'> b'breakcore'
<class 'numpy.bytes_'> b'breakcore'

 <class 'numpy.bytes_'> b'nu metal'
<class 'numpy.bytes_'> b'nu metal'

 <class 'numpy.bytes_'> b'disco'
<class 'nu

In [49]:
df = pd.DataFrame(data, columns=['artist_id', 'artist_name', 'artist_location', 'song_id', 'song_name', 'song_hottness','time_signature','artist_terms','artist_mbtags','mode','year'])


In [50]:
df

,artist_id,artist_name,artist_location,song_id,song_name,song_hottness,time_signature,artist_terms,artist_mbtags,mode,year
0,AREJXK41187B9A4ACC,b'Rapha\xc3\xabl',France,SOGSMXL12A81C23D88,Je Sais Que La Terre Est Plate,0.547953,4,"[b'chanson', b'visual kei', b'hip hop', b'pop ...",[],1,2008
1,AR2XRFQ1187FB417FE,b'Julie Zenatti',,SOMBCOW12AAF3B229F,On Efface,0.475638,4,"[b'chanson', b'dance pop', b'pop rock', b'soft...",[],0,2004
2,ARODOO01187FB44F4A,b'The Baltimore Consort',,SOEYIHF12AB017B5F4,Howells Delight,NaN,4,"[b'early music', b'celtic', b'mediaeval', b'fo...",[],1,0
3,ARJGW911187FB586CA,b'I Hate Sally',,SODJYEC12A8C13D757,Martha Served,NaN,3,"[b'post-hardcore', b'doomcore', b'metalcore', ...",[],1,2007
4,AR9HQ6Y1187FB3C2CB,b'Orlando Pops Orchestra',,SOGSOUE12A58A76443,Zip-A-Dee-Doo-Dah (Song of the South),NaN,4,"[b'orchestra', b'musical theater', b'british',...",[],1,0
...,...,...,...,...,...,...,...,...,...,...,...
95,ARMRCET1187FB52049,b'Fergie',"Hacienda Heights, CA",SOHKKXU12A67ADA08F,Voodoo Doll,0.670827,4,"[b'pop rap', b'california', b'funk', b'hip hop...",[],1,0
96,ARYLSGC11F43A69EF6,b'Los Amigos Invisibles',Venezuela,SOVKSAD12A8C142C13,Nerio Compra Una Contestadora,NaN,5,"[b'funk rock', b'chill-out', b'jungle music', ...",[],1,1998
97,ARAIABB1187B9AC6E2,b'Seal',"Paddington, London, England",SONUOBX12A8C141070,Killer (Single Version),0.221549,4,"[b'dance pop', b'neo soul', b'pop rock', b'urb...","[b'british', b'soul', b'pop', b'uk', b'english...",0,0
98,ARE54I01187FB3AE3D,b'Flink',,SOLVAZN12AB017DBAD,Sunset Manifesto,0.000000,4,"[b'alternative rock', b'indie rock', b'pop roc...",[],0,0


In [8]:
#triples

user_profiles = pd.read_csv('triplets_1000.txt', sep='\t', names = ['userID','songID', 'play_count'])
user_profiles

,userID,songID,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
...,...,...,...
995,5a905f000fc1ff3df7ca807d57edb608863db05d,SOYPJMP12AF72A901D,1
996,5a905f000fc1ff3df7ca807d57edb608863db05d,SOYRHNG12A8C14002E,1
997,5a905f000fc1ff3df7ca807d57edb608863db05d,SOYVSHP12A6702016E,2
998,5a905f000fc1ff3df7ca807d57edb608863db05d,SOYYYFE12A81C2395E,1


### Create the data set that is useful for the content based approach

In [39]:
df_new = df[['artist_id','song_id', 'time_signature', 'artist_terms', 'year']]

In [40]:
df_new

,artist_id,song_id,time_signature,artist_terms,year
0,AREJXK41187B9A4ACC,SOGSMXL12A81C23D88,4,"[b'chanson', b'visual kei', b'hip hop', b'pop ...",2008
1,AR2XRFQ1187FB417FE,SOMBCOW12AAF3B229F,4,"[b'chanson', b'dance pop', b'pop rock', b'soft...",2004
2,ARODOO01187FB44F4A,SOEYIHF12AB017B5F4,4,"[b'early music', b'celtic', b'mediaeval', b'fo...",0
3,ARJGW911187FB586CA,SODJYEC12A8C13D757,3,"[b'post-hardcore', b'doomcore', b'metalcore', ...",2007
4,AR9HQ6Y1187FB3C2CB,SOGSOUE12A58A76443,4,"[b'orchestra', b'musical theater', b'british',...",0
...,...,...,...,...,...
95,ARMRCET1187FB52049,SOHKKXU12A67ADA08F,4,"[b'pop rap', b'california', b'funk', b'hip hop...",0
96,ARYLSGC11F43A69EF6,SOVKSAD12A8C142C13,5,"[b'funk rock', b'chill-out', b'jungle music', ...",1998
97,ARAIABB1187B9AC6E2,SONUOBX12A8C141070,4,"[b'dance pop', b'neo soul', b'pop rock', b'urb...",0
98,ARE54I01187FB3AE3D,SOLVAZN12AB017DBAD,4,"[b'alternative rock', b'indie rock', b'pop roc...",0
